In [196]:
from imblearn.pipeline import Pipeline
import os
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PowerTransformer

In [197]:
cwd = os.getcwd()
# load data
fpath = "./data/interim/Combined_Player_Per90_Cleaned.csv"
if os.path.isfile(fpath) == False:
    print('Error - fpath is invalid')
else:
    df = pd.read_csv(fpath)

In [198]:
pd.set_option('display.max_columns', None) # reset when done viewing
df.head(10)

,Unnamed: 0,PlayerID,Season,League,Player,Nation,Pos,Squad,Age,Born,90s,Gls_shoot,Sh_shoot,SoT_shoot,SoT%_shoot,Sh/90_shoot,SoT/90_shoot,G/Sh_shoot,G/SoT_shoot,Dist_shoot,FK_shoot,PK_shoot,PKatt_shoot,xG_shoot,npxG_shoot,npxG/Sh_shoot,G-xG_shoot,np:G-xG_shoot,SCA_gsc,SCA90_gsc,PassLive_gsc,PassDead_gsc,Drib_gsc,Sh_gsc,Fld_gsc,Def_gsc,GCA_gsc,GCA90_gsc,PassLive.1_gsc,PassDead.1_gsc,Drib.1_gsc,Sh.1_gsc,Fld.1_gsc,Def.1_gsc,Total Cmp_pass,Total Att_pass,Total Cmp%_pass,Total TotDist_pass,Total PrgDist_pass,Short Cmp_pass,Short Att_pass,Short Cmp%_pass,Medium Cmp_pass,Medium Att_pass,Medium Cmp%_pass,Long Cmp_pass,Long Att_pass,Long Cmp%_pass,Ast_pass,xA_pass,A-xA_pass,KP_pass,1/3_pass,PPA_pass,CrsPA_pass,Prog_pass,Att_pass_type,Pass Types: Live_pass_type,Pass Types: Dead_pass_type,Pass Types: FK_pass_type,Pass Types: TB_pass_type,Pass Types: Press_pass_type,Pass Types: Sw_pass_type,Pass Types: Crs_pass_type,Pass Types: CK_pass_type,Corner Kicks: In_pass_type,Corner Kicks: Out_pass_type,Corner Kicks: Str_pass_type,Height: Ground_pass_type,Height: Low_pass_type,Height: High_pass_type,Body Parts: Left_pass_type,Body Parts: Right_pass_type,Body Parts: Head_pass_type,Body Parts: TI_pass_type,Body Parts: Other_pass_type,Outcomes: Cmp_pass_type,Outcomes: Off_pass_type,Outcomes: Out_pass_type,Outcomes: Int_pass_type,Outcomes: Blocks_pass_type,Tackles: Tkl_def,Tackles: TklW_def,Tackles: Def 3rd_def,Tackles: Mid 3rd_def,Tackles: Att 3rd_def,Vs Dribbles: Tkl_def,Vs Dribbles: Att_def,Vs Dribbles: Tkl%_def,Vs Dribbles: Past_def,Pressures: Press_def,Pressures: Succ_def,Pressures: %_def,Pressures: Def 3rd_def,Pressures: Mid 3rd_def,Pressures: Att 3rd_def,Blocks: Blocks_def,Blocks: Sh_def,Blocks: ShSv_def,Blocks: Pass_def,Int_def,Tkl+Int_def,Clr_def,Err_def,Touches_poss,Def Pen_poss,Def 3rd_poss,Mid 3rd_poss,Att 3rd_poss,Att Pen_poss,Live_poss,Succ_poss,Att_poss,Succ%_poss,#Pl_poss,Megs_poss,Carries_poss,TotDist_poss,PrgDist_poss,Prog_poss,1/3_poss,CPA_poss,Mis_poss,Dis_poss,Targ_poss,Rec_poss,Rec%_poss,Prog.1_poss,CrdY_misc,CrdR_misc,2CrdY_misc,Fls_misc,Fld_misc,Off_misc,Crs_misc,Int_misc,TklW_misc,PKwon_misc,PKcon_misc,OG_misc,Recov_misc,Won_misc,Lost_misc,Won%_misc,MP_time,Min_time,Mn/MP_time,Min%_time,Starts_time,Mn/Start_time,Compl_time,Subs_time,Mn/Sub_time,unSub_time,PPM_time,onG_time,onGA_time,+/-_time,+/-90_time,On-Off_time,onxG_time,onxGA_time,xG+/-_time,xG+/-90_time,On-Off.1_time,prev_season_league
0,0,11975017,2017-2018,Serie A,Allan,BRA,MF,Napoli,26,1991,31.7,0.13,1.17,0.38,32.4,1.17,0.38,0.11,0.33,21.0,0.0,0.0,0.0,0.14,0.14,0.00,-0.01,-0.01,3.06,3.06,2.33,0.03,0.19,0.09,0.22,0.19,0.47,0.47,0.38,0.0,0.00,0.00,0.03,0.06,49.15,57.44,85.6,770.22,196.31,28.17,31.23,90.2,16.47,18.99,86.7,3.88,5.62,69.1,0.16,0.14,0.02,1.01,5.74,1.61,0.06,5.36,57.44,56.62,0.82,0.22,0.16,9.81,0.79,0.69,0.13,0.03,0.00,0.06,43.31,7.44,6.69,5.27,49.75,1.55,0.47,0.28,49.15,0.19,0.82,1.23,1.17,3.50,2.52,0.73,2.08,0.69,0.85,3.25,26.2,2.40,29.18,9.21,31.6,5.14,15.43,8.61,1.74,0.06,0.0,1.67,0.60,4.10,0.44,0.00,70.63,1.29,7.82,43.56,24.61,1.67,69.87,2.30,3.82,1.90,2.46,0.13,56.18,240.32,134.48,8.14,2.78,0.47,1.64,2.15,58.17,52.68,2.86,4.23,0.06,0.00,0.0,1.20,1.51,0.00,0.69,0.60,2.52,0.0,0.0,0.0,9.34,0.28,1.01,22.0,38,2850,75,83.3,32,0,19,6,0,0,2.39,62,21,41,1.29,0.19,57.0,20.2,36.8,1.16,0.47,0
1,1,11975017,2018-2019,Serie A,Allan,BRA,MF,Napoli,27,1991,29.1,0.03,0.86,0.07,8.0,0.86,0.07,0.04,0.50,23.6,0.0,0.0,0.0,0.04,0.04,0.00,-0.00,-0.00,2.99,2.99,2.47,0.00,0.24,0.10,0.10,0.07,0.31,0.31,0.27,0.0,0.00,0.03,0.00,0.00,68.08,76.80,88.6,1257.90,339.93,29.14,31.99,91.1,29.42,32.03,91.8,8.93,11.13,80.2,0.10,0.11,-0.01,1.24,9.18,1.27,0.03,6.80,76.80,73.13,3.68,2.37,0.41,9.31,1.48,0.31,0.03,0.00,0.00,0.00,56.49,10.55,9.76,5.22,68.93,1.65,0.58,0.31,68.08,0.45,0.72,1.31,1.48,4.26,3.16,1.55,2.10,0.62,1.44,4.43,32.6,2.99,32.27,9.07,28.1,8.87,18.42,4.98,2.27,0.31,0.0,1.96,0.55,4.81,0.86,0.00,88.76,2.75,18.93,57.73,17.84,1.17,85.53,1.92,2.68,2.47,1.99,0.07,71.00,322.78,179.97,8.18,

In [200]:
# transform and standardize numeric features
numeric_transformer = Pipeline(steps=[('pt', PowerTransformer())])
numeric_features = list(df.columns[10:-1])
numeric_features.append(df.columns[8])

data_prep = ColumnTransformer(
    transformers=[
        ('numeric', numeric_transformer, numeric_features)])
model_df = pd.DataFrame(data_prep.fit_transform(df), columns=numeric_features)

In [203]:
# one hot encode league and prev_season_league

league = pd.get_dummies(df.League, prefix='League')
prev_league = pd.get_dummies(df.prev_season_league, prefix='Prev_League')
model_df = pd.concat([model_df, league, prev_league], axis=1)

In [204]:
# load cluster assignment df
fpath = "./data/interim/cluster_assignments.csv"
if os.path.isfile(fpath) == False:
    print('Error - fpath is invalid')
else:
    km_df = pd.read_csv(fpath)
    
# add cluster assignments to model_df
temp = df.merge(km_df, on=["Squad", "Season"])
model_df = pd.concat([model_df, foo["team_play_style"]], axis=1)

In [206]:
model_df.head(10)

,90s,Gls_shoot,Sh_shoot,SoT_shoot,SoT%_shoot,Sh/90_shoot,SoT/90_shoot,G/Sh_shoot,G/SoT_shoot,Dist_shoot,FK_shoot,PK_shoot,PKatt_shoot,xG_shoot,npxG_shoot,npxG/Sh_shoot,G-xG_shoot,np:G-xG_shoot,SCA_gsc,SCA90_gsc,PassLive_gsc,PassDead_gsc,Drib_gsc,Sh_gsc,Fld_gsc,Def_gsc,GCA_gsc,GCA90_gsc,PassLive.1_gsc,PassDead.1_gsc,Drib.1_gsc,Sh.1_gsc,Fld.1_gsc,Def.1_gsc,Total Cmp_pass,Total Att_pass,Total Cmp%_pass,Total TotDist_pass,Total PrgDist_pass,Short Cmp_pass,Short Att_pass,Short Cmp%_pass,Medium Cmp_pass,Medium Att_pass,Medium Cmp%_pass,Long Cmp_pass,Long Att_pass,Long Cmp%_pass,Ast_pass,xA_pass,A-xA_pass,KP_pass,1/3_pass,PPA_pass,CrsPA_pass,Prog_pass,Att_pass_type,Pass Types: Live_pass_type,Pass Types: Dead_pass_type,Pass Types: FK_pass_type,Pass Types: TB_pass_type,Pass Types: Press_pass_type,Pass Types: Sw_pass_type,Pass Types: Crs_pass_type,Pass Types: CK_pass_type,Corner Kicks: In_pass_type,Corner Kicks: Out_pass_type,Corner Kicks: Str_pass_type,Height: Ground_pass_type,Height: Low_pass_type,Height: High_pass_type,Body Parts: Left_pass_type,Body Parts: Right_pass_type,Body Parts: Head_pass_type,Body Parts: TI_pass_type,Body Parts: Other_pass_type,Outcomes: Cmp_pass_type,Outcomes: Off_pass_type,Outcomes: Out_pass_type,Outcomes: Int_pass_type,Outcomes: Blocks_pass_type,Tackles: Tkl_def,Tackles: TklW_def,Tackles: Def 3rd_def,Tackles: Mid 3rd_def,Tackles: Att 3rd_def,Vs Dribbles: Tkl_def,Vs Dribbles: Att_def,Vs Dribbles: Tkl%_def,Vs Dribbles: Past_def,Pressures: Press_def,Pressures: Succ_def,Pressures: %_def,Pressures: Def 3rd_def,Pressures: Mid 3rd_def,Pressures: Att 3rd_def,Blocks: Blocks_def,Blocks: Sh_def,Blocks: ShSv_def,Blocks: Pass_def,Int_def,Tkl+Int_def,Clr_def,Err_def,Touches_poss,Def Pen_poss,Def 3rd_poss,Mid 3rd_poss,Att 3rd_poss,Att Pen_poss,Live_poss,Succ_poss,Att_poss,Succ%_poss,#Pl_poss,Megs_poss,Carries_poss,TotDist_poss,PrgDist_poss,Prog_poss,1/3_poss,CPA_poss,Mis_poss,Dis_poss,Targ_poss,Rec_poss,Rec%_poss,Prog.1_poss,CrdY_misc,CrdR_misc,2CrdY_misc,Fls_misc,Fld_misc,Off_misc,Crs_misc,Int_misc,TklW_misc,PKwon_misc,PKcon_misc,OG_misc,Recov_misc,Won_misc,Lost_misc,Won%_misc,MP_time,Min_time,Mn/MP_time,Min%_time,Starts_time,Mn/Start_time,Compl_time,Subs_time,Mn/Sub_time,unSub_time,PPM_time,onG_time,onGA_time,+/-_time,+/-90_time,On-Off_time,onxG_time,onxGA_time,xG+/-_time,xG+/-90_time,On-Off.1_time,Age,League_Bundesliga,League_La Liga,League_Ligue 1,League_Premier League,League_Serie A,Prev_League_0,Prev_League_Bundesliga,Prev_League_La Liga,Prev_League_Ligue 1,Prev_League_Premier League,Prev_League_Serie A,team_play_style
0,1.163359,0.459118,0.279384,0.330566,0.302987,0.279297,0.330414,0.608622,0.593877,0.700368,-0.582571,-0.38418,-0.417829,0.552507,0.647864,-0.684785,-0.121771,-0.116832,0.974117,0.973858,1.127936,-0.387510,0.965331,0.312890,1.156519,1.833710,1.312527,1.312450,1.529173,-0.444283,-0.519359,-0.614242,1.319478,3.182989,0.893151,0.794734,0.961099,0.296159,-0.171677,1.743557,1.789198,0.443988,0.267504,0.220430,0.243985,-0.314194,-0.542828,0.706623,0.999869,0.919290,0.331101,0.448944,1.495201,1.293722,-0.539498,1.192094,0.794734,1.005965,-1.234596,-1.080462,1.076849,1.057793,-0.553013,-0.114543,0.188506,0.142666,-0.545420,2.072247,0.985960,0.497288,-0.453949,-0.308182,1.221961,-0.307170,0.025318,0.198787,0.893151,0.490005,0.075968,0.923595,0.149644,1.994105,2.301336,0.009270,2.548363,2.180010,0.924978,1.760120,-0.278799,1.899483,2.161209,3.005073,0.357647,0.206149,2.034813,1.551047,0.429924,-0.974631,-0.359472,0.938097,-0.375352,1.242027,-0.955434,-0.708145,0.909752,-0.658683,-0.757014,1.197197,0.977530,0.098313,1.098568,1.450764,1.429612,-0.687692,1.434555,1.351601,1.393142,0.708646,0.739383,1.475013,1.506210,0.765265,0.711646,1.211876,1.047466,1.169364,-0.867471,0.505680,-1.067048,-0.442595,-0.297984,0.083709,0.664701,-1.027956,-0.114543,-0.375352,2.301336,-0.483037,-0.557563,-0.278707,0.416098,-1.171750,-0.183378,-1.173129,1.729017,1.158543,-0.119046,1.056415,1.125073,-2.113634,0.502747,0.751779,-1.255206

In [207]:
# group lasso, linear regression